In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ['CUDA_VISIBLE_DEVICES'] = '2'
os.environ["TF_CPP_MIN_LOG_LEVEL"] = '3'
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as dataset
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
from copy import deepcopy as d_copy
import random
from torch.utils.tensorboard import SummaryWriter

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#error_index = 0
vgg16_bn = torchvision.models.vgg16_bn(pretrained=True)#.to(device)
vgg16_bn.eval()
print(device)

cuda


In [3]:
from importnb import Notebook

In [4]:
with Notebook(lazy=True):
    import F4F_model
header = F4F_model

In [5]:
# set randomness
seed = 0

header.set_randomness(seed) # ipynb module import means run all cell in file
# load dataset
dataset_path = "/media/2/Network/Imagenet_dup/"
retrain_model_path = "/media/0/Network/0821_to_fullmodels/"
batch_size = 32 # 32~ out of memory in 3080
num_train = 128000 #640000

train_dataloader,test_dataloader = header.get_dataset(num_train,batch_size,
                  dataset_path,retrain_model_path)

===INFO===
torch ver : 1.4.0
torchvision ver : 0.5.0 
GPU model : TITAN RTX
cuda
train dataset[1000], test dataset[391] are loaded
train dataset[4000], test dataset[1563] are loaded


In [6]:
#In_layer_number = 34 # 34 conv5_1 convolution
#Out_layer_number = 36 # 36 conv5_1 relu 
error_index=0
max_epoch = 10
num_error = 64

In [7]:
log_file = "./3fc_64_acc_log_0906.txt"

In [9]:
if torch.cuda.device_count() >1 :
    print("data parallel start")
    f4f = nn.DataParallel(f4f).to(device)

In [10]:
#split_model = split_layer(vgg16_bn,0,Out_layer_number)

original_model = d_copy(vgg16_bn).to(device)
# subset of vgg16 (whole layer) with f4f
#header.hook_register(vgg16_bn,error_index,num_error)
header.hook_register(vgg16_bn,num_error,error_index)
target_model = header.Target_model(vgg16_bn).to(device)



input  Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


In [11]:
# training
def training(f4f,target_model,original_model,
             train_dataloader,test_dataloader,batch_size,
             log_file,TensorBoardWriter,
             retrain_model_path,
             loss_fn,optimizer,
             num_error,
             max_epochs=20,subset=False):
    target_model.to(device)
    original_model.to(device)
    target_model.eval()
    original_model.eval()
    acc_log = []
    loss_log = []
    for epoch in range(1,max_epochs+1):
        running_loss = 0.0
        total_loss = []
        total_avg_loss = 0.0
        total = 0
        correct = 0
        f4f.train()
        # update f4f filter
        #target_model.apply_f4f(f4f,error_info)
    
        # compare
        for i, data in enumerate(train_dataloader):
            error_index = i % (512-num_error)
            error_info = header.make_error_info(error_index,num_error).to(device)
            
            
            if i % 10 == 0:
                print(".",end="")
            inputs,labels = data
            inputs,labels = inputs.to(device), labels.to(device)
            
            #target_out = target_model(inputs,f4f,error_info)
            result = target_model(inputs,f4f,error_info)
            target_out,origin_weight, replace_weight = result
            
            
            _,predicted = torch.max(target_out,1) # target_out.data : no grad, target_out : with grad
            
            total += labels.size(0)
            correct += (predicted==labels).sum().item()
            #loss = loss_fn(target_out,labels)
            
            loss = loss_fn(0.5,target_out,labels,
                            origin_weight, replace_weight) # check ratio is same of eval func
            
            running_loss += loss.item()
            #target_model.model.zero_grad() # might be useless
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            if i % 100 == 99: 
                total_loss.append(running_loss/100)
                print("")
                print('[%d, %5d] loss: %.6f' % (epoch+1, i+1, running_loss/(100*batch_size) )) 
                print("[%d, %5d] acc : %.6f"%(epoch+1,i, 100*correct/total))
                running_loss = 0.0
        # save weight
        #print((len(train_dataloader)/batch_size))
        if len(total_loss) != 0:
            total_avg_loss = sum(total_loss)/(len(total_loss)*batch_size)
        acc = 100*correct/total
        acc_log.append(acc)
        loss_log.append(total_avg_loss)
        if total_avg_loss != 0:
            print("total average loss : %.3f" %(total_avg_loss))
        else :
            print("total loss :" ,total_loss)
        print("==epoch %d ==  train acc : %.4f" %(epoch,acc))
        TensorBoardWriter.add_scalar("Model1/ACC_Train",acc,epoch)
        TensorBoardWriter.add_scalar("Model1/LOSS_Train",total_avg_loss,epoch)
        acc = header.eval(target_model,test_dataloader,epoch,loss_fn,batch_size,
                   f4f,error_info,log_file,TensorBoardWriter)
        
    return acc_log,loss_log
                

In [12]:

#loss_fn = nn.CrossEntropyLoss()
def loss_fn(loss1_ratio,pred,label, filter_orig,filter_f4f):
    if(loss1_ratio <0 or loss1_ratio >1 ):
        print("wrong parameter ratio ",loss1_ratio)
        return nan
    loss1 = nn.CrossEntropyLoss()
    loss2 = nn.MSELoss()
    a = loss1(pred,label)
    b = loss2(filter_orig,filter_f4f)
    return loss1_ratio * a + (1-loss1_ratio) * b

In [13]:
lr_list = []
for i in range(-8,0,1):
    for j in range(1,3):
        lr_list.append(j/2*10 **(i))

In [14]:

#optimizer = torch.optim.SGD(param_list,lr=0.01,weight_decay=1e-4)
f = open(log_file,"w")
f.close()
total_acc_log = []
total_loss_log = []
for lr in lr_list: 
    
    writer = SummaryWriter("./runs/F4F_model_lr_"+str(lr))
    target_model = header.Target_model(vgg16_bn).to(device)
    f4f = header.F4F().to(device)
    optimizer = torch.optim.SGD(f4f.parameters(),lr=lr,weight_decay=1e-4) 

    acc_log,loss_log= training(f4f,target_model,original_model,
                  train_dataloader,test_dataloader,batch_size,
                  log_file,writer,retrain_model_path,
                  loss_fn,optimizer,
                  num_error,max_epoch,True)
    f = open(log_file,"w")
    print("last acc : %.4f & loss : %4.f in lr "%(acc_log[-1],loss_log[-1]),lr,file=f)
    total_acc_log.append(acc_log)
    total_loss_log.append(loss_log)
    f.close()
        
writer.close()


..........
[2,   100] loss: 29.667704
[2,    99] acc : 1.468750
..........
[2,   200] loss: 243.782298
[2,   199] acc : 0.828125
..........
[2,   300] loss: 533.110750
[2,   299] acc : 0.562500
..........
[2,   400] loss: 813.019061
[2,   399] acc : 0.445312
..........
[2,   500] loss: 1105.113060
[2,   499] acc : 0.381250
..........
[2,   600] loss: 1381.735883
[2,   599] acc : 0.333333
..........
[2,   700] loss: 1665.878250
[2,   699] acc : 0.308036
..........
[2,   800] loss: 1959.457808
[2,   799] acc : 0.281250
..........
[2,   900] loss: 2275.010618
[2,   899] acc : 0.256944
..........
[2,  1000] loss: 2535.799694
[2,   999] acc : 0.237500
..........
[2,  1100] loss: 2827.852571
[2,  1099] acc : 0.221591
..........
[2,  1200] loss: 3024.278003
[2,  1199] acc : 0.216146
..........
[2,  1300] loss: 3308.003618
[2,  1299] acc : 0.213942
..........
[2,  1400] loss: 3569.394514
[2,  1399] acc : 0.207589
..........
[2,  1500] loss: 3800.883518
[2,  1499] acc : 0.197917
..........
[2, 

KeyboardInterrupt: 

In [ ]:

w = 10
h = 10
cols = 32
rows = 16
def feature_print(pic):
    #print("test with 'after pooling 4 feature'")
    fig = plt.figure(figsize=(64,32))
    ax = []
    for i in range(cols*rows):
        ch = pic[i,:,:]
        ax.append(fig.add_subplot(rows,cols,i+1))
        ax[-1].set_title(str(i)+"th ch (14x14)")
        plt.imshow(ch)

In [ ]:
feature_print(first_feature[0])